## *1. Installing required modules*

 ##  1. INSTALLING AND IMPORTING REQUIRED MODULES

In [17]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
!pip install PyAudio

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Asus\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-kx6pbqqb\\pyaudio_028d8a9449284eea89edbe82aa1cb982\\setup.py'"'"'; __file__='"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-kx6pbqqb\\pyaudio_028d8a9449284eea89edbe82aa1cb982\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Asus\AppData\Local\Temp\pip-wheel-sg8y5nwk'
       cwd: C:\Users\Asus\AppData\Local\Temp\pip-install-kx6pbqqb\pyaudio_028d8a9449284eea89edbe82aa1cb982\
  Complete output (9 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  copying s


  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio: started
    Running setup.py install for PyAudio: finished with status 'error'


In [19]:
!pip install pydub

In [20]:
!pip install speechrecognition

In [21]:
!pip install pyLDAvis

In [22]:
!pip install datasets

In [23]:
!pip install plotly

## *2. Importing required modules*

In [30]:
#import pyaudio
import wave
from pydub import AudioSegment
import speech_recognition as sr
from io import BytesIO
from base64 import b64decode
from IPython.display import Javascript
import IPython.display as ipd
#import spacy
import numpy as np
import nltk
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from datasets import load_dataset
import pandas as pd
%matplotlib inline
import plotly.express as px
#Libraries for preprocessing
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
#Download once if using NLTK for preprocessing
import nltk
nltk.download('punkt')
#Libraries for vectorisation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import LatentDirichletAllocation
#Libraries for clustering
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

C:\Users\Asus\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


# **3. DATASET FOR TOPIC MODELING**

In [31]:
#loading dataset
dataset = load_dataset('medical_questions_pairs')
dataset

Using custom data configuration default


Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Dataset medical_questions_pairs downloaded and prepared to C:\Users\Asus\.cache\huggingface\datasets\medical_questions_pairs\default\0.0.0\db30a35b934dceb7abed5ef6b73a432bb59682d00e26f9a1acd960635333bc80. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 3048
    })
})

In [32]:
#creating dataframe from dataset
df = pd.DataFrame(dataset)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3048 entries, 0 to 3047
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   train   3048 non-null   object
dtypes: object(1)
memory usage: 23.9+ KB


In [33]:
#overview of first 5 records
df.head(5)

,train
0,"{'dr_id': 1, 'question_1': 'After how many hou..."
1,"{'dr_id': 1, 'question_1': 'After how many hou..."
2,"{'dr_id': 1, 'question_1': 'Am I over weight (..."
3,"{'dr_id': 1, 'question_1': 'Am I over weight (..."
4,"{'dr_id': 1, 'question_1': 'Aspirin allergy - ..."


In [34]:
#creating csv from dataframe
df.to_csv("medical_data.csv")

In [35]:
#creating text file from dataframe
with open("medical_dataset.txt", "w") as f:
  for i in df['train']:
    line = list(str(i).split("'"))
    f.write(str(line[5]) + "\n" + str(line[9]) + "\n")

# **4. TOPIC MODELLING with LDA**

## *1. Text cleaning*

We use the following function to clean our texts and return a list of tokens:

In [ ]:

#spacy.load('en')
spacy.load("en_core_web_sm")
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

We use NLTK’s Wordnet to find the meanings of words, synonyms, antonyms, and more. In addition, we use WordNetLemmatizer to get the root word.

In [ ]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

Filter out stop words:

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

Now we can define a function to prepare the text for topic modelling:

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

Open up our data, read line by line, for each line, prepare text for LDA, then add to a list.

Now we can see how our text data are converted:

In [ ]:
import random
text_data = []
with open('medical_dataset.txt') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)